In [1]:
import findspark
spark_path = '../../2016UCSD/255/spark-1.6.0-bin-hadoop2.6/'
findspark.init(spark_path)
from pyspark import SparkContext
sc = SparkContext(master="local[3]")


In [2]:
from get_features import *
import os, pickle
from sklearn import linear_model, datasets
domain = 'random70'
#domain = '100'
train_file = 'Data/%s/trainData' % (domain)
test_file = 'Data/%s/testData' % (domain)

pickled_trainX = 'trainX%s.pickle' % (domain)
pickled_trainY = 'trainY%s.pickle' % (domain)
pickled_testX = 'testX%s.pickle' % (domain)
pickled_testY = 'testY%s.pickle' % (domain)

In [6]:
def getData(infile):
    lines = [ line for line in open(infile, 'r').readlines()]
    text, Y = [line.split(',')[0] for line in lines], \
                        [int(line.split(',')[1]) for line in lines]
    print len(text), len(Y)
    assert(len(text) == len(Y), "we've got a problem")
    wp = word_processor()
    rdd = sc.parallelize(text).map(lambda x: wp.get_features1(x))
    data = rdd.collect()
    assert(len(data) == len(Y), "we've got a problem2")
    return [data, Y]

<ipython-input-6-4e841e63d2ac>:5: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(text) == len(Y), "we've got a problem")
<ipython-input-6-4e841e63d2ac>:9: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(data) == len(Y), "we've got a problem2")


In [7]:
if not os.path.exists(pickled_testX) or not os.path.exists(pickled_testY):
    testX, testY = getData(test_file)
    pickle.dump(testX, open(pickled_testX, 'w'))
    pickle.dump(testY, open(pickled_testY, 'w'))
else:
    testX = pickle.load(open(pickled_testX, 'r'))
    testY = pickle.load(open(pickled_testY, 'r'))

600 600


In [9]:
if not os.path.exists(pickled_trainX) or not os.path.exists(pickled_trainY):
    trainX, trainY = getData(train_file)
    pickle.dump(trainX, open(pickled_trainX, 'w'))
    pickle.dump(trainY, open(pickled_trainY, 'w'))
else:
    trainX = pickle.load(open(pickled_trainX, 'r'))
    trainY = pickle.load(open(pickled_trainY, 'r'))

1400 1400


In [32]:
# for x, y in zip(trainX, trainY):
#     print x, y
def model1(trainX, trainY, testX, testY):
    logistic = linear_model.LogisticRegression(C=1e5)
    logistic = linear_model.LogisticRegression(C=10)
    #logistic = linear_model.LogisticRegression(C = 1)
    trainX, trainY, testX, testY = np.array(trainX), np.array(trainY), \
                                        np.array(testX), np.array(testY)
    print trainX.shape, trainY.shape,testX.shape,testY.shape  
    logistic.fit(trainX, trainY)
    print 'train score =', 'test score' 
    print logistic.score(trainX, trainY), logistic.score(testX, testY)
    return logistic

In [26]:
model1(trainX, trainY, testX, testY)

(1400, 165) (1400,) (600, 165) (600,)
train score = test score
0.612142857143 0.578333333333


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
domain = 'random70_all'
#domain = '100'
train_file = 'Data/%s/trainData' % (domain)
test_file = 'Data/%s/testData' % (domain)

pickled_trainX = 'trainX%s.pickle' % (domain)
pickled_trainY = 'trainY%s.pickle' % (domain)
pickled_testX = 'testX%s.pickle' % (domain)
pickled_testY = 'testY%s.pickle' % (domain)
if not os.path.exists(pickled_trainX) or not os.path.exists(pickled_trainY):
    trainX, trainY = getData(train_file)
    pickle.dump(trainX, open(pickled_trainX, 'w'))
    pickle.dump(trainY, open(pickled_trainY, 'w'))
else:
    trainX = pickle.load(open(pickled_trainX, 'r'))
    trainY = pickle.load(open(pickled_trainY, 'r'))

8982 8982


In [28]:
if not os.path.exists(pickled_testX) or not os.path.exists(pickled_testY):
    testX, testY = getData(test_file)
    pickle.dump(testX, open(pickled_testX, 'w'))
    pickle.dump(testY, open(pickled_testY, 'w'))
else:
    testX = pickle.load(open(pickled_testX, 'r'))
    testY = pickle.load(open(pickled_testY, 'r'))

3848 3848


In [33]:
model1(trainX, trainY, testX, testY)

(8982, 165) (8982,) (3848, 165) (3848,)
train score = test score
0.591627699844 0.574324324324


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)